# ベクトル化された小説データに対するSVMを用いた「腐向け」タグ分類

In [1]:
import gensim
import json
import numpy as np
from sklearn import svm

In [2]:
model = gensim.models.doc2vec.Doc2Vec.load('novel2vec.model')

In [3]:
top_tags = json.load(open('top-tag-ids.json'))

In [4]:
target_novel_ids = top_tags['腐向け']

In [5]:
X = []
y = []
ids = []
for index, vector in enumerate(model.docvecs):
    X.append(vector)
    novel_id = model.docvecs.index_to_doctag(index)
    ids.append(novel_id)
    if int(novel_id) in target_novel_ids:
        y.append(1)
    else:
        y.append(0)

In [6]:
from sklearn import cross_validation
X_train, X_test, y_train, y_test, ids_train, ids_test = cross_validation.train_test_split(X, y, ids, test_size=0.2, random_state=10)

In [7]:
%time clf = svm.SVC(kernel='linear').fit(X_train, y_train)

CPU times: user 12min 21s, sys: 252 ms, total: 12min 21s
Wall time: 12min 22s


In [8]:
from sklearn.externals import joblib
joblib.dump(clf, 'fumuke-classifier.pkl')

['fumuke-classifier.pkl',
 'fumuke-classifier.pkl_01.npy',
 'fumuke-classifier.pkl_02.npy',
 'fumuke-classifier.pkl_03.npy',
 'fumuke-classifier.pkl_04.npy',
 'fumuke-classifier.pkl_05.npy',
 'fumuke-classifier.pkl_06.npy',
 'fumuke-classifier.pkl_07.npy',
 'fumuke-classifier.pkl_08.npy',
 'fumuke-classifier.pkl_09.npy',
 'fumuke-classifier.pkl_10.npy',
 'fumuke-classifier.pkl_11.npy']

In [9]:
clf.score(X_test, y_test)

0.86135833863106592

In [10]:
y_predict = clf.predict(X_test)

In [11]:
import collections
collections.Counter(y_test)

Counter({0: 16259, 1: 2617})

In [12]:
collections.Counter(y_predict)

Counter({0: 18876})

In [13]:
false_negatives = []
false_positives = []
for index, predict in enumerate(y_predict):
    test = y_test[index]
    if test == 0 and predict == 1:
        false_positives.append(ids_test[index])
    if test == 1 and predict == 0:
        false_negatives.append(ids_test[index])

In [14]:
len(false_positives)

0

In [15]:
len(false_negatives)

2617

In [16]:
np.random.choice(false_positives, 10, replace=False)

ValueError: a must be non-empty

In [ ]:
np.random.choice(false_negatives, 10, replace=False)